In [2]:
pip install transformers

     |████████████████████████████████| 3.8 MB 14.2 MB/s 
     |████████████████████████████████| 895 kB 73.6 MB/s 
     |████████████████████████████████| 6.5 MB 70.2 MB/s 
     |████████████████████████████████| 596 kB 74.7 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [43]:
import os
import json
import random
import numpy as np
import pandas as pd

from tqdm import tqdm

import torch
from torch.optim import Adam, AdamW
from torch.utils.data import TensorDataset, RandomSampler, DataLoader
from transformers import BertTokenizer, BertTokenizerFast, BertForQuestionAnswering
from transformers import get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import  f1_score

import gc

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
%cd /content/gdrive/My Drive/NLP

In [4]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2022-03-19 14:03:25--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M  --.-KB/s    in 0.1s    

2022-03-19 14:03:26 (313 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2022-03-19 14:03:26--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [5]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [6]:
def data_load(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = [] 
    start_ids = []
    end_ids = []

    for datas in squad_dict['data']:
        for paragraphs in datas['paragraphs']:
            context = paragraphs['context']
            for qas in paragraphs['qas']:
                question = qas['question']
                for answer in qas['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer['text'])

                    start_index = answer['answer_start']
                    start_ids.append(start_index)

                    answer['text'] = answer['text'].rstrip()
                    
                    end_index = start_index + len(answer['text'])
                    end_ids.append(end_index)                  

    return pd.DataFrame({'contexts' : contexts, 'questions' : questions, 'answers' : answers, 'start_ids': start_ids,'end_ids': end_ids})

In [18]:
train_df = data_load('KorQuAD_v1.0_train.json')
valid_df = data_load('KorQuAD_v1.0_dev.json')

In [26]:
def qa_preprocess(df, batch_size=16, method='train'):
  if method == 'train' or method == 'valid':    
      batch_input = tokenizer(df['contexts'].tolist(), df['questions'].tolist(), truncation=True, padding=True)

      start_ids = df['start_ids'].tolist()
      end_ids = df['end_ids'].tolist()

      start_positions = [batch_input.char_to_token(i, start_ids[i]) for i in range(len(start_ids))]
      end_positions = [batch_input.char_to_token(i, end_ids[i]-1) for i in range(len(end_ids))]
      deleting_list = [i for i, v in enumerate(end_positions) if v == None]
          
      batch_input.update({'start_positions': start_positions, 'end_positions': end_positions})

     
      batch_input = {key : np.delete(np.array(value), deleting_list, axis=0) for key, value in batch_input.items()}
      batch_input = {key : torch.tensor(value.astype(int)) for key, value in batch_input.items()}

      dataset = TensorDataset(batch_input['input_ids'], batch_input['attention_mask'], batch_input['token_type_ids'], batch_input['start_positions'], batch_input['end_positions'])
      if method == 'train':
        dataset_sampler = RandomSampler(dataset)
        dataloader = DataLoader(dataset, sampler=dataset_sampler, batch_size=batch_size)
      elif method == 'valid':
        dataloader = DataLoader(dataset, batch_size=batch_size)
      return dataloader, deleting_list

  elif method == 'test':
      batch_input = tokenizer(df['contexts'].tolist(), df['questions'].tolist(), truncation=True, padding=True)
      batch_input = {key : torch.tensor(value) for key, value in batch_input.items()}

      dataset = TensorDataset(batch_input['input_ids'], batch_input['attention_mask'], batch_input['token_type_ids'])
      dataloader = DataLoader(dataset, batch_size=batch_size)

  return dataloader

# model training


In [15]:
def train_one_epoch(optimizer, scheduler, dataloader):
    model.train()

    train_loss = 0.0

    for batchs in tqdm(dataloader):
        batch = tuple(b.to(device) for b in batchs)

        inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
                "start_positions": batch[3],
                "end_positions": batch[4],
            }


        optimizer.zero_grad()

        outputs = model(**inputs)
        
        loss = outputs[0]
        
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        train_loss += loss.item()

    avg_train_loss = train_loss / len(dataloader)
    
    return avg_train_loss

# model_evaluate

In [14]:
def evaluate_one_epoch(dataloader):
    model.eval()

    start_preds = []
    end_preds = []
    input = []

    for batchs in tqdm(dataloader):
        batch = tuple(b.to(device) for b in batchs)

        inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2]
            }

        with torch.no_grad():
            outputs = model(**inputs)

        # CPU로 데이터 이동
        start_pred = outputs['start_logits'].detach().cpu()
        end_pred = outputs['end_logits'].detach().cpu()

        input.append(inputs['input_ids'].detach().cpu())
        start_preds.append(start_pred)
        end_preds.append(end_pred)

    input = torch.cat(input, dim=0).tolist()
    start_preds = torch.cat(start_preds, dim=0).argmax(dim=-1).tolist()
    end_preds = torch.cat(end_preds, dim=0).argmax(dim=-1).tolist()

    answer = [tokenizer.decode(input[s:e+1]) for input, s, e in zip(input,start_preds,end_preds)]

    return answer

# QA Model

In [ ]:
def qa_model(train_data, dev_data,lr=1e-4,epochs = 4, batch_size=32, bert='klue/bert-base', save=True, path='bert_qa'):
    gc.collect()
    torch.cuda.empty_cache()

    global tokenizer
    tokenizer = BertTokenizerFast.from_pretrained(bert)

    global model,device
    model = BertForQuestionAnswering.from_pretrained(bert)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    train_dataloader,train_delete = qa_preprocess(train_data, batch_size=batch_size, method = 'train')
    val_dataloader,val_delete = qa_preprocess(dev_data, batch_size=batch_size, method = 'valid')
    print('')
    print('Preprocess Compelete')
    print('')

    # no_decay = ['bias', 'LayerNorm.weight']
    # optimizer_grouped_parameters = [
    #     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
    #         'weight_decay': 0.01},
    #     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    # ]

    optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)

    total_steps = len(train_dataloader) * epochs

    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0,
                                                num_training_steps = total_steps)

    print('')
    print('Model Training Start')
    print(f'Epochs : {epochs} / Learning Rate : {str(lr)} / Batch Size : {batch_size}')
    print('')

    loss = []
    f1 = []
    for epoch in range(1,epochs+1):
        print(f"epoch = {epoch}")

        loss = train_one_epoch(optimizer, scheduler, dataloader=train_dataloader)
        preds = evaluate_one_epoch(dataloader=val_dataloader)

        f1_score = f1_score(valid_df.drop(val_delete,axis=0)['answers'].tolist(),preds,average='micro')
        f1.append(f1_score)

        print('')

        if save:
            model.save_pretrained(f'models/{path}')
            tokenizer.save_pretrained(f'models/{path}')
            print('Model Save')
    
    print('')
    print("Training Complete!")

    return {'loss':loss,'f1 score':f1}

In [ ]:
score,preds = qa_model(train_df, valid_df, lr = 1e-5,epochs=1,batch_size=16, bert='klue/bert-base')

# Test

In [45]:
def QA_test(test_data, batch_size=32,bert='QA'):
    global tokenizer
    tokenizer = BertTokenizer.from_pretrained(f'klue/bert-base')
    test_dataloader = qa_preprocess(test_data, batch_size=batch_size, method='test')

    global model, device
    model = BertForQuestionAnswering.from_pretrained(f'models/{bert}')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    answer = evaluate_one_epoch(dataloader=test_dataloader)

    new_df = valid_df[['contexts','questions']].copy()
    new_df['answers'] = answer
    
    return  new_df

In [46]:
df = QA_test(valid_df, batch_size=32,bert='QA')

100%|██████████| 181/181 [00:54<00:00,  3.31it/s]


In [56]:
sample = df.sample(5)

for i in range(len(sample)):
    print('context : ', sample['contexts'].tolist()[i])
    print('questions : ', sample['questions'].tolist()[i])
    print('answers : ', sample['answers'].tolist()[i])
    print('')

context :  고종의 정비로 1871년 첫 왕자를 5일 만에 잃고, 최익현 등과 손잡고 흥선대원군의 간섭을 물리치고 고종의 친정을 유도했다. 민씨 척족을 기용함으로써 세도정권을 부활시켰으며, 1882년 임오군란 이후 일본한테 겨냥하는 견제를 위해 청나라의 지원에 의존하다가 1894년 청일전쟁에서 청나라가 패배당한 이후에는 러시아를 끌어들여 일본을 견제했다. 맨 처음에는 개항에 미온적이었으나, 점진적인 개화시책을 통해 친일 성향을 띤 급진 개화파의 개화정책에 제동을 걸었다. 그러다가 흥선대원군과 주조선 일본공사 미우라 고로의 공모에 의해 일본인 병사와 낭인들에게 암살당했다(을미사변(乙未事變), 1895년). 사후 대한제국이 성립되면서 황후로 추봉되었다. 정식 시호는 효자원성정화합천홍공성덕제휘열목명성태황후(孝慈元聖正化合天洪功誠德齊徽烈穆明成太皇后)이다.
questions :  명성황후가 암살당한 해는?
answers :  1895년

context :  페니키아(그리스어: Φοινίκη, 라틴어: Phœnicia)는 고대 가나안의 북쪽에 근거지를 둔 고대 문명이다. 중심 지역은 오늘날의 레바논과 시리아, 이스라엘 북부로 이어지는 해안에 있었다. 페니키아 문명은 기원전 1200년경에서 900년경까지 지중해를 가로질러 퍼져나간 진취적인 해상 무역 문화를 가졌다고 과거의 고대사엔 수록되어 있지만. 최근의 고고학 발굴로 페니키아 문명은 그보다 훨씬 오래전인 기원전 40세기에 '기시 문명'이 지중해 문명과 메소포타미아를 연계하는 문명이라고 한다. 학자들은 이 '기시 문명(Gish)'이 아프리카의 '쿠마 문화(Kuma), 나일 강 유역의 쿠시 문화(Kush)와 연결고리가 있으며 그 관계를 연구중이라 한다. 연구된 바에 의하면 북아프리카의 튀니지(고대 카르타고)를 중심으로 알제리, 리비아 등 북아프리카권역의 국가와 무역으로 이탈리아의 중, 남부 도시에 식민지를 건설하였으며 리베리아(현 스페인)의 항구도시에 식민지를 건설하였다는 것은 이집트 기록에 의한 것이다. 고대의 경계